In [1]:
from importlib import reload
import utils; reload(utils)
from utils import *
data_path =  '/Volumes/Backups/data/LUNA16/patches/'

Using cuDNN version 5105 on context None
Preallocating 2867/4095 Mb (0.700000) on cuda0
Mapped name None to device cuda0: GeForce GTX 650 (0000:01:00.0)
Using Theano backend.


In [2]:
%matplotlib inline
import os, random
from shutil import copytree
data_path =  '/Volumes/Backups/data/LUNA16/patches/'

In [3]:
theano.config.dnn.conv.precision = 'float32'
print(theano.config.dnn.conv.precision)

float32


In [8]:
for i in range(80):
    patches1 = random.choice(os.listdir("/Volumes/Backups/data/LUNA16/patches/patches1/"))
    patches0 = random.choice(os.listdir("/Volumes/Backups/data/LUNA16/patches/patches0/"))
    copytree(data_path + 'patches1/' + patches1, '/Volumes/Backups/data/LUNA16/patches/sample/patches1/' + patches1)
    copytree(data_path + 'patches0/' + patches0, '/Volumes/Backups/data/LUNA16/patches/sample/patches0/' + patches0)
    

FileExistsError: [Errno 17] File exists: '/Volumes/Backups/data/LUNA16/patches/sample/patches0/1.3.6.1.4.1.14519.5.2.1.6279.6001.837810280808122125183730411210_435706_224x224.bc'

In [7]:
import numpy as np
np.set_printoptions(precision=4,linewidth=100)
import pandas as pd
from vgg16bn import Vgg16BN

In [ ]:
model = vgg_ft_bn(2)

In [ ]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(2)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(1e-3),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
from skimage import color

In [ ]:
np.sctypes

In [ ]:
np_img = np.zeros((1,224,224))

In [ ]:
np_img


In [ ]:
import glob2
from skimage import color
image_labels = []
img_array = np.zeros((1,3,224,224),dtype=np.float32)
print(type(img_array[0][0][0][0]))
i=0
#for filename in glob2.glob(data_path+'sample/*/*'):
for filename in (os.listdir(data_path + "sample/")):
    print(filename)
    for image in  glob2.glob(data_path + "sample/" + filename+'/*'):
        if (filename == 'patches0'):
            image_labels.append(np.int16(0.0))
            print(type(image_labels[0]))
        else:
            image_labels.append(np.int16(1.0))
        print ("imagename: " + image)
        img = color.gray2rgb(load_array(image)).astype(np.float32)
        print("img type: " + str(type(img[0][0][0])))
        np_img = np.transpose(np.asarray(img,dtype=np.float32),(2,0,1))
        print(np_img.shape)
        print("np_img type : " + str(type(np_img[0][0][0])))
        tmp_img = np.zeros((1,3,224,224),dtype=np.float32)
        print(tmp_img.shape)
        print("tmp_img type : " + str(type(tmp_img[0][0][0][0])))
        tmp_img[0] = np_img
        print("np.shape: " + str(tmp_img.shape))
        img_array = np.concatenate((img_array, tmp_img))
        print("img_array.shape :" + str(img_array.shape))
        i += 1
img_array = img_array[1:]



In [ ]:
print(type(np_img[0][0][0]))

In [ ]:
 img_array.shape

In [ ]:
img.shape

In [ ]:
type(img[0][0][0])

In [ ]:
plt.imshow(img_array[13][2],cmap=plt.cm.gray)

In [ ]:
type(img_array[0][0][0][0])

In [ ]:
len(image_labels)

In [ ]:
type(image_labels[19])

In [ ]:
ohe_image_labels = to_categorical(np.array(np.float32(image_labels))).astype(np.float32)

In [ ]:
type(ohe_image_labels[0][0])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trn_labels, val_labels = train_test_split(ohe_image_labels, test_size=0.2, random_state=42)

In [ ]:
trn,val = train_test_split(img_array, test_size=0.2, random_state=42,)

In [ ]:
trn.shape

In [ ]:
type(val[0][0][1][0])

In [ ]:
batch_size = 4

In [ ]:
type(val[1][0][1][0])

In [ ]:
model.fit(trn, 
          trn_labels, 
          batch_size=batch_size, 
          nb_epoch=1, 
          validation_data=(val, val_labels),verbose = 1)


In [ ]:
model.save_weights(data_path+'results/ft1.h5')

In [ ]:
conv_layers,fc_layers = split_at(model, Convolution2D)

In [ ]:
conv_layers

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
%time conv_trn_feat = conv_model.predict(trn)

In [ ]:
conv_trn_feat.shape

In [ ]:
%time conv_val_feat = conv_model.predict(val)

In [ ]:
conv_val_feat.shape

In [ ]:
save_array(data_path+'results/conv_val_feat.dat', conv_val_feat)

In [ ]:
type(conv_trn_feat[0][0][0][0])


In [ ]:
nf=128; p=0

In [ ]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((2,2)),
        Convolution2D(2,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]


In [ ]:
lrg_model = Sequential(get_lrg_layers())

In [ ]:
lrg_model.input_shape, lrg_model.output_shape

In [ ]:
lrg_model.summary()

In [ ]:
lrg_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
lrg_model.fit(conv_trn_feat, trn_labels, batch_size=8, nb_epoch=1, 
              validation_data=(conv_val_feat, val_labels))


In [ ]:
lrg_model.optimizer.lr = 0.00001

In [ ]:
lrg_model.fit(conv_trn_feat, trn_labels, batch_size=8, nb_epoch=10, 
              validation_data=(conv_val_feat, val_labels))


In [ ]:
unresArr198 = load_array('/Volumes/Backups/data/unlabeled_patient_collapsed_images-198x3x224x224.bc')

In [ ]:
unlabeledPredictionvgg = lrg_model.predict(unresArr198, batch_size=8, verbose=1)

In [ ]:
#conv_feat = load_array(data_root + 'results/conv_feat.dat')
#conv_val_feat = load_array(data_root + 'results/conv_val_feat.dat')

In [ ]:
print(theano.config)